# LSTM

In [ ]:
# Install required packages
%pip install --upgrade pip
%pip install polars numpy scikit-learn matplotlib joblib openpyxl fastexcel tensorflow

# For TensorFlow on Mac, you need to install tensorflow-macos
%pip install tensorflow-macos tensorflow-metal

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement tensorflow-macos (from versions: none)
ERROR: No matching distribution found for tensorflow-macos
Note: you may need to restart the kernel to use updated packages.


In [4]:
import polars as pl
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Concatenate, BatchNormalization
import matplotlib.pyplot as plt
import os
from joblib import Parallel, delayed
from datetime import timedelta
import openpyxl

# Configuración de Matplotlib para evitar errores con Tkinter
import matplotlib
matplotlib.use('TkAgg')

## Constantes

Definicón de las constantes y rutas.

In [ ]:
# Definición de la ruta del proyecto
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
SUBJECTS_RELATIVE_PATH = "data/Subjects"
SUBJECTS_PATH = os.path.join(PROJECT_ROOT, SUBJECTS_RELATIVE_PATH)
# Crear directorio para resultados si no existe
FIGURES_DIR = os.path.join(PROJECT_ROOT, "figures", "lstm")
os.makedirs(FIGURES_DIR, exist_ok=True)
MODELS_DIR = os.path.join(PROJECT_ROOT, "models")
os.makedirs(MODELS_DIR, exist_ok=True)

subject_files = [f for f in os.listdir(SUBJECTS_PATH) if f.startswith("Subject") and f.endswith(".xlsx")]
print(f"Total sujetos: {len(subject_files)}")

Total sujetos: 54


## Preprocesamiento y Procesamiento de Datos

In [15]:
def get_cgm_window(bolus_time, cgm_df: pl.DataFrame, window_hours: int = 2) -> np.ndarray:
    """
    Obtiene la ventana de datos CGM para un tiempo de bolo específico.

    Parámetros:
    -----------
    bolus_time : datetime
        Tiempo del bolo de insulina
    cgm_df : pl.DataFrame
        DataFrame con datos CGM
    window_hours : int, opcional
        Horas de la ventana de datos (default: 2)

    Retorna:
    --------
    np.ndarray
        Ventana de datos CGM o None si no hay suficientes datos
    """
    window_start = bolus_time - timedelta(hours=window_hours)
    window = cgm_df.filter(
        (pl.col("date") >= window_start) & (pl.col("date") <= bolus_time)
    ).sort("date").tail(24)
    
    if window.height < 24:
        return None
    return window.get_column("mg/dl").to_numpy()

def calculate_iob(bolus_time, basal_df: pl.DataFrame, half_life_hours: float = 4.0) -> float:
    """
    Calcula la insulina activa en el cuerpo (IOB).

    Parámetros:
    -----------
    bolus_time : datetime
        Tiempo del bolo de insulina
    basal_df : pl.DataFrame
        DataFrame con datos de insulina basal
    half_life_hours : float, opcional
        Vida media de la insulina en horas (default: 4.0)

    Retorna:
    --------
    float
        Cantidad de insulina activa
    """
    if basal_df is None or basal_df.is_empty():
        return 0.0
    
    iob = 0.0
    for row in basal_df.iter_rows(named=True):
        start_time = row["date"]
        duration_hours = row["duration"] / (1000 * 3600)
        end_time = start_time + timedelta(hours=duration_hours)
        rate = row["rate"] if row["rate"] is not None else 0.9
        
        if start_time <= bolus_time <= end_time:
            time_since_start = (bolus_time - start_time).total_seconds() / 3600
            remaining = rate * (1 - (time_since_start / half_life_hours))
            iob += max(0.0, remaining)
    return iob

def process_subject(subject_path: str, idx: int) -> list:
    """
    Procesa los datos de un sujeto.

    Parámetros:
    -----------
    subject_path : str
        Ruta al archivo del sujeto
    idx : int
        Índice del sujeto

    Retorna:
    --------
    list
        Lista de diccionarios con características procesadas
    """
    print(f"Procesando {os.path.basename(subject_path)} ({idx+1}/{len(SUBJECTS_PATH)})...")
    
    try:
        cgm_df = pl.read_excel(subject_path, sheet_name="CGM")
        bolus_df = pl.read_excel(subject_path, sheet_name="Bolus")
        try:
            basal_df = pl.read_excel(subject_path, sheet_name="Basal")
        except Exception:
            basal_df = None
    except Exception as e:
        print(f"Error al cargar {os.path.basename(subject_path)}: {e}")
        return []

    # Conversión de fechas
    cgm_df = cgm_df.with_columns(pl.col("date").cast(pl.Datetime))
    bolus_df = bolus_df.with_columns(pl.col("date").cast(pl.Datetime))
    if basal_df is not None:
        basal_df = basal_df.with_columns(pl.col("date").cast(pl.Datetime))
    
    cgm_df = cgm_df.sort("date")

    processed_data = []
    for row in bolus_df.iter_rows(named=True):
        bolus_time = row["date"]
        cgm_window = get_cgm_window(bolus_time, cgm_df)
        
        if cgm_window is not None:
            iob = calculate_iob(bolus_time, basal_df)
            hour_of_day = bolus_time.hour / 23.0
            bg_input = row["bgInput"] if row["bgInput"] is not None else cgm_window[-1]
            normal = row["normal"] if row["normal"] is not None else 0.0
            
            # Cálculo del factor de sensibilidad personalizado
            isf_custom = 50.0
            if normal > 0 and bg_input > 100:
                isf_custom = (bg_input - 100) / normal
            
            features = {
                'subject_id': idx,
                'cgm_window': cgm_window,
                'carbInput': row["carbInput"] if row["carbInput"] is not None else 0.0,
                'bgInput': bg_input,
                'insulinCarbRatio': row["insulinCarbRatio"] if row["insulinCarbRatio"] is not None else 10.0,
                'insulinSensitivityFactor': isf_custom,
                'insulinOnBoard': iob,
                'hour_of_day': hour_of_day,
                'normal': normal
            }
            processed_data.append(features)
    
    return processed_data

# Ejecución en paralelo
all_processed_data = Parallel(n_jobs=-1)(
    delayed(process_subject)(
        os.path.join(SUBJECTS_PATH, f), 
        idx
    ) for idx, f in enumerate(subject_files)
)

all_processed_data = [item for sublist in all_processed_data for item in sublist]

# Conversión a DataFrame
df_processed = pl.DataFrame(all_processed_data)
print("Muestra de datos procesados combinados:")
print(df_processed.head())
print(f"Total muestras: {len(df_processed)}")

Procesando Subject21.xlsx (1/69)...
Procesando Subject37.xlsx (2/69)...
Procesando Subject17.xlsx (3/69)...
Procesando Subject40.xlsx (4/69)...
Procesando Subject6.xlsx (5/69)...
Procesando Subject7.xlsx (6/69)...
Procesando Subject41.xlsx (7/69)...
Procesando Subject16.xlsx (8/69)...
Procesando Subject36.xlsx (9/69)...
Procesando Subject20.xlsx (10/69)...
Procesando Subject11.xlsx (11/69)...
Procesando Subject46.xlsx (12/69)...
Procesando Subject50.xlsx (13/69)...
Procesando Subject27.xlsx (14/69)...
Procesando Subject31.xlsx (15/69)...
Procesando Subject30.xlsx (16/69)...
Procesando Subject26.xlsx (17/69)...
Procesando Subject1.xlsx (18/69)...
Procesando Subject51.xlsx (19/69)...
Procesando Subject47.xlsx (20/69)...
Procesando Subject10.xlsx (21/69)...
Procesando Subject29.xlsx (22/69)...
Procesando Subject2.xlsx (23/69)...
Procesando Subject52.xlsx (24/69)...
Procesando Subject44.xlsx (25/69)...
Procesando Subject13.xlsx (26/69)...


Could not determine dtype for column 5, falling back to string


Procesando Subject33.xlsx (27/69)...
Procesando Subject25.xlsx (28/69)...
Procesando Subject48.xlsx (29/69)...
Procesando Subject49.xlsx (30/69)...
Procesando Subject24.xlsx (31/69)...
Procesando Subject32.xlsx (32/69)...
Procesando Subject12.xlsx (33/69)...
Procesando Subject45.xlsx (34/69)...
Procesando Subject53.xlsx (35/69)...
Procesando Subject3.xlsx (36/69)...
Procesando Subject28.xlsx (37/69)...
Procesando Subject35.xlsx (38/69)...
Procesando Subject23.xlsx (39/69)...
Procesando Subject8.xlsx (40/69)...
Procesando Subject19.xlsx (41/69)...
Procesando Subject39.xlsx (42/69)...
Procesando Subject4.xlsx (43/69)...
Procesando Subject54.xlsx (44/69)...
Procesando Subject42.xlsx (45/69)...
Procesando Subject15.xlsx (46/69)...
Procesando Subject14.xlsx (47/69)...
Procesando Subject43.xlsx (48/69)...
Procesando Subject5.xlsx (49/69)...
Procesando Subject38.xlsx (50/69)...
Procesando Subject18.xlsx (51/69)...
Procesando Subject9.xlsx (52/69)...
Procesando Subject22.xlsx (53/69)...
Proces

### Ordenamiento y Limpieza

Verificación de los valores nulos.

In [16]:
# División de ventana CGM y otras características
cgm_columns = [f'cgm_{i}' for i in range(24)]
df_cgm = pl.DataFrame({
    col: [row['cgm_window'][i] for row in all_processed_data]
    for i, col in enumerate(cgm_columns)
}, schema={col: pl.Float64 for col in cgm_columns})

# Combinar con otras características
df_final = pl.concat([
    df_cgm,
    df_processed.drop('cgm_window')
], how="horizontal")

# Verificar valores nulos
print("Verificación de valores nulos en df_final:")
df_final = df_final.drop_nulls()
print(df_final.null_count())

Verificación de valores nulos en df_final:
shape: (1, 32)
┌───────┬───────┬───────┬───────┬───┬──────────────────────┬────────────────┬─────────────┬────────┐
│ cgm_0 ┆ cgm_1 ┆ cgm_2 ┆ cgm_3 ┆ … ┆ insulinSensitivityFa ┆ insulinOnBoard ┆ hour_of_day ┆ normal │
│ ---   ┆ ---   ┆ ---   ┆ ---   ┆   ┆ ctor                 ┆ ---            ┆ ---         ┆ ---    │
│ u32   ┆ u32   ┆ u32   ┆ u32   ┆   ┆ ---                  ┆ u32            ┆ u32         ┆ u32    │
│       ┆       ┆       ┆       ┆   ┆ u32                  ┆                ┆             ┆        │
╞═══════╪═══════╪═══════╪═══════╪═══╪══════════════════════╪════════════════╪═════════════╪════════╡
│ 0     ┆ 0     ┆ 0     ┆ 0     ┆ … ┆ 0                    ┆ 0              ┆ 0           ┆ 0      │
└───────┴───────┴───────┴───────┴───┴──────────────────────┴────────────────┴─────────────┴────────┘


### Normalización de los Datos

In [17]:
# Normalizar características
scaler_cgm = MinMaxScaler(feature_range=(0, 1))
scaler_other = StandardScaler()

# Normalizar CGM y reshape para LSTM
X_cgm = scaler_cgm.fit_transform(df_final.select(cgm_columns).to_numpy())
X_cgm = X_cgm.reshape(X_cgm.shape[0], X_cgm.shape[1], 1)

# Normalizar otras características (incluyendo hour_of_day)
other_features = ['carbInput', 'bgInput', 'insulinOnBoard', 'insulinCarbRatio', 
                 'insulinSensitivityFactor', 'subject_id', 'hour_of_day']
X_other = scaler_other.fit_transform(df_final.select(other_features).to_numpy())

# Etiquetas
y = df_final.get_column('normal').to_numpy()

# Verificar NaN
print("\nVerificación de valores NaN:")
print(f"NaN en X_cgm: {np.isnan(X_cgm).sum()}")
print(f"NaN en X_other: {np.isnan(X_other).sum()}")
print(f"NaN en y: {np.isnan(y).sum()}")

if np.isnan(X_cgm).sum() > 0 or np.isnan(X_other).sum() > 0 or np.isnan(y).sum() > 0:
    raise ValueError("Valores NaN detectados en X_cgm, X_other o y")



Verificación de valores NaN:
NaN en X_cgm: 0
NaN en X_other: 0
NaN en y: 0


## División por Sujeto

In [18]:
# División por sujeto
subject_ids = df_final.get_column('subject_id').unique().to_numpy()
train_subjects, temp_subjects = train_test_split(subject_ids, test_size=0.2, random_state=42)
val_subjects, test_subjects = train_test_split(temp_subjects, test_size=0.5, random_state=42)

train_mask = np.isin(df_final.get_column('subject_id').to_numpy(), train_subjects)
val_mask = np.isin(df_final.get_column('subject_id').to_numpy(), val_subjects)
test_mask = np.isin(df_final.get_column('subject_id').to_numpy(), test_subjects)

X_cgm_train, X_cgm_val, X_cgm_test = X_cgm[train_mask], X_cgm[val_mask], X_cgm[test_mask]
X_other_train, X_other_val, X_other_test = X_other[train_mask], X_other[val_mask], X_other[test_mask]
y_train, y_val, y_test = y[train_mask], y[val_mask], y[test_mask]
subject_test = df_final.filter(pl.lit(test_mask)).get_column('subject_id').to_numpy()

print("\nFormas de los conjuntos de datos:")
print(f"Entrenamiento CGM: {X_cgm_train.shape}, Validación CGM: {X_cgm_val.shape}, Prueba CGM: {X_cgm_test.shape}")
print(f"Entrenamiento Otros: {X_other_train.shape}, Validación Otros: {X_other_val.shape}, Prueba Otros: {X_other_test.shape}")
print(f"Sujetos de prueba: {test_subjects}")


Formas de los conjuntos de datos:
Entrenamiento CGM: (33272, 24, 1), Validación CGM: (2743, 24, 1), Prueba CGM: (8636, 24, 1)
Entrenamiento Otros: (33272, 7), Validación Otros: (2743, 7), Prueba Otros: (8636, 7)
Sujetos de prueba: [ 5 19 32 13 48 49]


## Modelo LSTM

In [19]:
# Modelo LSTM
def create_lstm_model(cgm_shape: tuple, other_features_shape: tuple) -> Model:
    """
    Crea un modelo LSTM con dos entradas.
    
    Parámetros:
    -----------
    cgm_shape : tuple
        Forma de los datos CGM
    other_features_shape : tuple
        Forma de otras características
        
    Retorna:
    --------
    Model
        Modelo LSTM compilado
    """
    # Definir las entradas
    cgm_input = Input(shape=cgm_shape[1:], name='cgm_input')
    other_input = Input(shape=(other_features_shape[1],), name='other_input')

    # Capas LSTM apiladas para procesar CGM
    lstm_out = LSTM(128, return_sequences=True)(cgm_input)
    lstm_out = LSTM(64, return_sequences=False)(lstm_out)
    lstm_out = BatchNormalization()(lstm_out)
    lstm_out = Dropout(0.2)(lstm_out)

    # Combinar con otras características
    combined = Concatenate()([lstm_out, other_input])

    # Capas densas
    dense = Dense(64, activation='relu')(combined)
    dense = BatchNormalization()(dense)
    dense = Dropout(0.2)(dense)
    output = Dense(1, activation='linear')(dense)

    # Crear y compilar modelo
    model = Model(inputs=[cgm_input, other_input], outputs=output)
    
    return model

# Función de pérdida personalizada
def custom_mse(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    """
    Función de pérdida MSE personalizada que penaliza más las sobrepredicciones.
    
    Parámetros:
    -----------
    y_true : tf.Tensor
        Valores reales
    y_pred : tf.Tensor
        Valores predichos
        
    Retorna:
    --------
    tf.Tensor
        Valor de pérdida
    """
    error = y_true - y_pred
    overprediction_penalty = tf.where(error < 0, 2 * tf.square(error), tf.square(error))
    return tf.reduce_mean(overprediction_penalty)

### Creación del Modelo

In [20]:
# Crear y compilar modelo
model = create_lstm_model(X_cgm_train.shape, X_other_train.shape)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=custom_mse
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cgm_input           │ (None, 24, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 24, 128)   │     66,560 │ cgm_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     49,408 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 64)        │        256 │ lstm_1[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ other_input         │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 71)        │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ other_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,608 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 121,153 (473.25 KB)

 Trainable params: 120,897 (472.25 KB)

 Non-trainable params: 256 (1.00 KB)

## Entrenamiento del Modelo

In [21]:
# Entrenar modelo
print("\nEntrenando modelo LSTM...")
history = model.fit(
    [X_cgm_train, X_other_train],
    y_train,
    validation_data=([X_cgm_val, X_other_val], y_val),
    epochs=100,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ],
    verbose=1
)


Entrenando modelo LSTM...
Epoch 1/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 65s 56ms/step - loss: 12.2659 - val_loss: 22.8280
Epoch 2/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 110s 105ms/step - loss: 4.4280 - val_loss: 30.1002
Epoch 3/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 80s 77ms/step - loss: 3.4670 - val_loss: 2.7306
Epoch 4/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 90s 87ms/step - loss: 3.0654 - val_loss: 5.1043
Epoch 5/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 87s 83ms/step - loss: 2.9835 - val_loss: 2.8937
Epoch 6/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 84s 80ms/step - loss: 2.8829 - val_loss: 1.1392
Epoch 7/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 76s 73ms/step - loss: 2.7584 - val_loss: 3.7294
Epoch 8/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 79s 76ms/step - loss: 2.7061 - val_loss: 336.7675
Epoch 9/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 79s 76ms/step - loss: 2.5463 - val_loss: 48.2544
Epoch 10/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 77s 74ms/step - loss: 2.6647 - val_loss: 40.7251
Epoch 11/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 73s 70ms/

## Evaluación del Modelo

In [22]:
# Evaluación del modelo
print("\nEvaluación del modelo LSTM:")
y_pred = model.predict([X_cgm_test, X_other_test]).flatten()

# Limpiar datos para métricas
def clean_predictions(y_true, y_pred):
    """
    Limpia predicciones eliminando valores infinitos o NaN.
    
    Parámetros:
    -----------
    y_true : np.ndarray
        Valores reales
    y_pred : np.ndarray
        Valores predichos
        
    Retorna:
    --------
    tuple
        Valores limpios (y_true, y_pred) y número de valores eliminados
    """
    mask = np.isfinite(y_pred) & np.isfinite(y_true)
    return y_true[mask], y_pred[mask], np.sum(~mask)

# Calcular métricas
y_test_clean, y_pred_clean, dropped = clean_predictions(y_test, y_pred)
print(f"\nValores eliminados en la evaluación: {dropped}")

mae = mean_absolute_error(y_test_clean, y_pred_clean)
rmse = np.sqrt(mean_squared_error(y_test_clean, y_pred_clean))
r2 = r2_score(y_test_clean, y_pred_clean)

print(f"MAE LSTM: {mae:.2f} u. de insulina")
print(f"RMSE LSTM: {rmse:.2f} u. de insulina")
print(f"R² LSTM: {r2:.2f}")


Evaluación del modelo LSTM:
270/270 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step

Valores eliminados en la evaluación: 0
MAE LSTM: 1.14 u. de insulina
RMSE LSTM: 2.02 u. de insulina
R² LSTM: 0.28


### Evaluación por Sujeto

In [23]:
# Evaluación por sujeto
print("\nRendimiento por sujeto:")
for subject_id in test_subjects:
    mask = subject_test == subject_id
    y_test_sub = y_test[mask]
    y_pred_sub = y_pred[mask]
    
    # Limpiar datos del sujeto
    y_test_sub_clean, y_pred_sub_clean, dropped_sub = clean_predictions(y_test_sub, y_pred_sub)
    
    if len(y_test_sub_clean) > 0:
        mae_sub = mean_absolute_error(y_test_sub_clean, y_pred_sub_clean)
        rmse_sub = np.sqrt(mean_squared_error(y_test_sub_clean, y_pred_sub_clean))
        r2_sub = r2_score(y_test_sub_clean, y_pred_sub_clean)
        print(
            f"Sujeto {subject_id}: "
            f"MAE={mae_sub:.2f}, "
            f"RMSE={rmse_sub:.2f}, "
            f"R²={r2_sub:.2f}, "
            f"Valores eliminados={dropped_sub}"
        )
    else:
        print(f"Sujeto {subject_id}: No hay suficientes datos válidos")


Rendimiento por sujeto:
Sujeto 5: MAE=0.81, RMSE=1.52, R²=-0.38, Valores eliminados=0
Sujeto 19: MAE=2.52, RMSE=3.49, R²=-0.94, Valores eliminados=0
Sujeto 32: MAE=0.53, RMSE=0.83, R²=0.86, Valores eliminados=0
Sujeto 13: MAE=0.54, RMSE=0.63, R²=0.45, Valores eliminados=0
Sujeto 48: MAE=1.03, RMSE=1.39, R²=0.06, Valores eliminados=0
Sujeto 49: MAE=3.89, RMSE=5.52, R²=-0.44, Valores eliminados=0


## Visualización

In [24]:
# Visualización del entrenamiento
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Pérdida del Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de la Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida MSE Personalizada')
plt.legend()
plt.title('Historial de Entrenamiento LSTM')
plt.savefig(os.path.join(FIGURES_DIR, 'training_history.png'), dpi=300, bbox_inches='tight')
plt.close()

# Visualización de predicciones
plt.figure(figsize=(12, 5))

invalid command name "7779437888process_stream_events"
    while executing
"7779437888process_stream_events"
    ("after" script)


<Figure size 1200x500 with 0 Axes>

### Predicciones vs Real

In [25]:
# Predicciones vs valores reales
plt.subplot(1, 2, 1)
plt.scatter(y_test_clean, y_pred_clean, alpha=0.5, label='LSTM')
plt.plot([0, 15], [0, 15], 'r--')
plt.xlabel('Dosis Real (u. de insulina)')
plt.ylabel('Dosis Predicha (u. de insulina)')
plt.legend()
plt.title('Predicción vs. Real (LSTM)')

Text(0.5, 1.0, 'Predicción vs. Real (LSTM)')

### Distribución Residual

In [26]:
# Distribución de residuos
plt.subplot(1, 2, 2)
residuals = y_test_clean - y_pred_clean
plt.hist(residuals, bins=20, alpha=0.5, label='Residuos LSTM')
plt.xlabel('Residuo (u. de insulina)')
plt.ylabel('Frecuencia')
plt.legend()
plt.title('Distribución de Residuos (LSTM)')

Text(0.5, 1.0, 'Distribución de Residuos (LSTM)')

In [27]:
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'predictions.png'), dpi=300, bbox_inches='tight')
plt.close()

invalid command name "7757709504process_stream_events"
    while executing
"7757709504process_stream_events"
    ("after" script)


## Guardado del Modelo

In [ ]:
# Guardar el modelo
model.save(os.path.join(MODELS_DIR, 'lstm_model.h5'))
print(f"\nModelo guardado en: {os.path.join(MODELS_DIR, 'lstm_model.keras')}")


Modelo guardado en: /Users/tomas/Desktop/FIUBA/TPP/TPP_Deep_Learning_Models/models/lstm_model.h5


: 